# Huggingface SageMaker-SDK - BERT Japanese QA example

1. [Introduction](#Introduction)  
2. [Development Environment and Permissions](#Development-Environment-and-Permissions)
    1. [Installation](#Installation)  
    2. [Permissions](#Permissions)
    3. [Uploading data to sagemaker_session_bucket](#Uploading-data-to-sagemaker_session_bucket)  
3. [(Optional) Deepen your understanding of SQuAD](#(Optional)-Deepen-your-understanding-of-SQuAD)    
4. [Fine-tuning & starting Sagemaker Training Job](#Fine-tuning-\&-starting-Sagemaker-Training-Job)  
    1. [Creating an Estimator and start a training job](#Creating-an-Estimator-and-start-a-training-job)  
    2. [Estimator Parameters](#Estimator-Parameters)   
    3. [Download fine-tuned model from s3](#Download-fine-tuned-model-from-s3)
    4. [Question Answering on Local](#Question-Answering-on-Local)  
5. [_Coming soon_:Push model to the Hugging Face hub](#Push-model-to-the-Hugging-Face-hub)

# Introduction

このnotebookはHuggingFaceの[run_squad.py](https://github.com/huggingface/transformers/blob/master/examples/legacy/question-answering/run_squad.py)を日本語データで動作する様に変更を加えたものです。    
データは[運転ドメインQAデータセット](https://nlp.ist.i.kyoto-u.ac.jp/index.php?Driving%20domain%20QA%20datasets)を使用します。    

このデモでは、AmazonSageMakerのHuggingFace Estimatorを使用してSageMakerのトレーニングジョブを実行します。    

_**NOTE: このデモは、SagemakerNotebookインスタンスで動作検証しています**_    
    _**データセットは各自許諾に同意の上ダウンロードしていただけますようお願いいたします（データサイズは約4MBです）**_

# Development Environment and Permissions

## Installation

このNotebookはSageMakerの`conda_pytorch_p36`カーネルを利用しています。    
日本語処理のため、`transformers`ではなく`transformers[ja]`をインスールします。

**_Note: このnotebook上で推論テストを行う場合、（バージョンが古い場合は）pytorchのバージョンアップが必要になります。_**

In [ ]:
# localで推論のテストを行う場合
!pip install torch==1.7.1

In [ ]:
!pip install "sagemaker>=2.31.0" "transformers[ja]==4.6.1" "datasets[s3]==1.6.2" --upgrade

## Permissions

ローカル環境でSagemakerを使用する場合はSagemakerに必要な権限を持つIAMロールにアクセスする必要があります。[こちら](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)を参照してください

In [ ]:
import sagemaker


sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

# データの準備

事前にデータ(`DDQA-1.0.tar.gz`)をこのnotobookと同じ階層に配置してください

以下では、データをダウンロードして解凍 (unzip) します。

In [ ]:
# Unzip

!tar -zxvf DDQA-1.0.tar.gz

## Uploading data to `sagemaker_session_bucket`

S3へデータをアップロードします。

In [ ]:
s3_prefix = 'samples/datasets/driving-domain-qa'

input_train = sess.upload_data(
    path='./DDQA-1.0/RC-QA/DDQA-1.0_RC-QA_train.json', 
    key_prefix=f'{s3_prefix}/train'
)

input_validation = sess.upload_data(
    path='./DDQA-1.0/RC-QA/DDQA-1.0_RC-QA_dev.json', 
    key_prefix=f'{s3_prefix}/valid'
)

In [ ]:
# データのUpload path

print(input_train)
print(input_validation)

# (Optional) Deepen your understanding of SQuAD

**このセクションはオプションであり、Fine-tuning & starting Sagemaker Training Jobまでスキップできます**

## 運転ドメインQAデータセットについて

運転ドメインQAデータセットはSQuAD2.0形式となっており、`run_squad.py`でそのまま実行できます。    
トレーニングジョブの実行とは関連しませんが、ここでは少しデータについて理解を深めたいと思います。

QAデータセットの形式(README_ja.txt)
--------------------

本QAデータセットの形式はSQuAD2.0と同じです。SQuAD2.0の問題は、「文章」、「質問」、「答え」の三つ組になっており、「答え」は「文章」の中の一部になっています。一部の問題は、「文章」の中に「答え」が無いなど、答えられない問題になっています。詳細は以下の論文をご参照ください。

Pranav Rajpurkar, Robin Jia, and Percy Liang.
Know what you don’t know: Unanswerable questions for SQuAD,
In ACL2018, pages 784–789.
https://www.aclweb.org/anthology/P18-2124.pdf

以下に、jsonファイル中のQAデータセットを例示します。    
注）jsonファイル中の"context"は「文章」

```json
{
    "version": "v2.0",
    "data": [
        {
            "title": "運転ドメイン",
            "paragraphs": [
                {
                    "context": "著者は以下の文章を書きました。本日お昼頃、梅田方面へ自転車で出かけました。ちょっと大きな交差点に差し掛かりました。自転車にまたがった若い女性が信号待ちしています。その後で私も止まって信号が青になるのを待っていました。",
                    "qas": [
                        {
                            "id": "55604556390008_00",
                            "question": "待っていました、の主語は何か？",
                            "answers": [
                                {
                                    "text": "私",
                                    "answer_start": 85
                                },
                                {
                                    "text": "著者",
                                    "answer_start": 0
                                }
                            ],
                            "is_impossible": false
                        }
                    ]
                }
            ]
        }
    ]
}
```

参考文献
--------

高橋 憲生、柴田 知秀、河原 大輔、黒橋 禎夫
ドメインを限定した機械読解モデルに基づく述語項構造解析
言語処理学会 第25回年次大会 発表論文集 (2019年3月)
　https://www.anlp.jp/proceedings/annual_meeting/2019/pdf_dir/B1-4.pdf    
　　※データセットの構築方法について記載

Norio Takahashi, Tomohide Shibata, Daisuke Kawahara and Sadao Kurohashi.
Machine Comprehension Improves Domain-Specific Japanese Predicate-Argument Structure Analysis,
In Proceedings of 2019 Conference on Empirical Methods in Natural Language Processing and 9th International Joint Conference on Natural Language Processing, Workshop MRQA: Machine Reading for Question Answering, 2019.
　https://mrqa.github.io/assets/papers/42_Paper.pdf    
　　※データセットの構築方法、文章中に答えが無い問題について記載

In [ ]:
# データの読み込み
import json

with open("./DDQA-1.0/RC-QA/DDQA-1.0_RC-QA_train.json", "r") as f:
    squad = json.load(f)

In [ ]:
squad['data'][0]['paragraphs'][0]

SQuAD2.0形式は少し複雑なjson形式となっています。    
次に`run_squad.py`内でどのような前処理が実行されているかについて少し触れます。    

このparagraphsにはコンテキストが1つと質問が2つ、回答が6つ含まれていますが、後の処理ではここから    
**2つの「コンテキスト」、「質問」、「答え」の三つ組**が作成されます。    
回答は1番目のものが使用されます。

In [ ]:
from transformers.data.processors.squad import SquadV2Processor
from transformers import squad_convert_examples_to_features

data_dir = './DDQA-1.0/RC-QA'
train_file = 'DDQA-1.0_RC-QA_train.json'

max_seq_length = 384 # トークン化後の最大入力シーケンス長。これより長いシーケンスは切り捨てられ、これより短いシーケンスはパディングされます
doc_stride = 128 # 長いドキュメントをチャンクに分割する場合、チャンク間でどのくらいのストライドを取るか
max_query_length = 64 # 質問のトークンの最大数。 これより長い質問はこの長さに切り捨てられます
threads = 1

In [ ]:
from transformers import AutoTokenizer

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking') 

In [ ]:
# jsonファイルを読みこみ、複雑な構造を分解します

processor = SquadV2Processor()
examples = processor.get_train_examples(data_dir, filename=train_file)

In [ ]:
# QuestionAnsweringモデルへ入力できるようにトークナイズします
# 以下の実行に数分時間がかかります

features, dataset = squad_convert_examples_to_features(
    examples=examples,
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    doc_stride=doc_stride,
    max_query_length=max_query_length,
    is_training=True,
    return_dataset="pt",
    threads=threads,
)

`dataset`は後に`dataloader`に渡され、以下のように使用されます。


```python
for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
    for step, batch in enumerate(epoch_iterator):

        # Skip past any already trained steps if resuming training
        if steps_trained_in_current_epoch > 0:
            steps_trained_in_current_epoch -= 1
            continue

        model.train()
        batch = tuple(t.to(args.device) for t in batch)

        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2],
            "start_positions": batch[3],
            "end_positions": batch[4],
        }
```

`input_ids`, `attention_mask`, `token_type_ids`はTransformerベースのモデルで一般的な入力形式です    
QuestionAnsweringモデル特有のものとして`start_positions`, `end_positions`が挙げられます

In [ ]:
# 参考に一つ目の中身を見てみます

i = 0
dataset[i]

In [ ]:
# すでに テキスト→トークン化→ID化されているため、逆の操作で元に戻します。
# 質問と文章が含まれていることが確認できます

tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(dataset[i][0]))

In [ ]:
# ID化→トークン化まで

tokenizer.convert_ids_to_tokens(dataset[i][0])

In [ ]:
# 回答は、start_positionsのトークンで始まり、end_positionsでトークンで終わるように表現されます
# 試しに該当箇所のトークンを文字に戻してみます。

print(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens([dataset[i][0][dataset[i][3]]])))
print(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens([dataset[i][0][dataset[i][4]]])))

これから実行する`QuestionAnswering`は、**「コンテキスト」**内から**「質問」**に対する**「答え」**となる`start_positions`と`end_positions`を予測し、そのスパンを抽出するタスクとなります。

# Fine-tuning & starting Sagemaker Training Job

`HuggingFace`のトレーニングジョブを作成するためには`HuggingFace` Estimatorが必要になります。    
Estimatorは、エンドツーエンドのAmazonSageMakerトレーニングおよびデプロイタスクを処理します。 Estimatorで、どのFine-tuningスクリプトを`entry_point`として使用するか、どの`instance_type`を使用するか、どの`hyperparameters`を渡すかなどを定義します。


```python
huggingface_estimator = HuggingFace(
    entry_point='train.py',
    source_dir='./scripts',
    base_job_name='huggingface-sdk-extension',
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    transformers_version='4.4',
    pytorch_version='1.6',
    py_version='py36',
    role=role,
    hyperparameters={
        'epochs': 1,
        'train_batch_size': 32,
        'model_name':'distilbert-base-uncased'
    }
)
```

SageMakerトレーニングジョブを作成すると、SageMakerは`huggingface`コンテナを実行するために必要なec2インスタンスの起動と管理を行います。    
Fine-tuningスクリプト`train.py`をアップロードし、`sagemaker_session_bucket`からコンテナ内の`/opt/ml/input/data`にデータをダウンロードして、トレーニングジョブを実行します。


```python
/opt/conda/bin/python train.py --epochs 1 --model_name distilbert-base-uncased --train_batch_size 32
```

`HuggingFace estimator`で定義した`hyperparameters`は、名前付き引数として渡されます。

またSagemakerは、次のようなさまざまな環境変数を通じて、トレーニング環境に関する有用なプロパティを提供しています。

* `SM_MODEL_DIR`：トレーニングジョブがモデルアーティファクトを書き込むパスを表す文字列。トレーニング後、このディレクトリのアーティファクトはモデルホスティングのためにS3にアップロードされます。

* `SM_NUM_GPUS`：ホストで使用可能なGPUの数を表す整数。

* `SM_CHANNEL_XXXX`：指定されたチャネルの入力データを含むディレクトリへのパスを表す文字列。たとえば、HuggingFace estimatorのfit呼び出しで`train`と`test`という名前の2つの入力チャネルを指定すると、環境変数`SM_CHANNEL_TRAIN`と`SM_CHANNEL_TEST`が設定されます。

このトレーニングジョブをローカル環境で実行するには、`instance_type='local'`、GPUの場合は`instance_type='local-gpu'`で定義できます。    
**_Note：これはSageMaker Studio内では機能しません_**

In [ ]:
# requirements.txtはトレーニングジョブの実行前に実行されます（コンテナにライブラリを追加する際に使用します）
# 残念なことにSageMakerのHuggingFaceコンテナは日本語処理（トークナイズ）に必要なライブラリが組み込まれていません
# したがってtransformers[ja]==4.6.1をジョブ実行前にインストールしています（fugashiとipadic）でも構いません
# tensorboardも組み込まれていないため、インストールします

!pygmentize ./scripts/requirements.txt

In [ ]:
# トレーニングジョブで実行されるコード
!pygmentize ./scripts/run_squad.py

In [ ]:
from sagemaker.huggingface import HuggingFace


# hyperparameters, which are passed into the training job
hyperparameters={
    'model_type': 'bert',
    'model_name_or_path': 'cl-tohoku/bert-base-japanese-whole-word-masking',
    'output_dir': '/opt/ml/model',
    'data_dir':'/opt/ml/input/data',
    'train_file': 'train/DDQA-1.0_RC-QA_train.json',
    'predict_file': 'validation/DDQA-1.0_RC-QA_dev.json',
    'version_2_with_negative': 'True',
    'do_train': 'True',
    'do_eval': 'True',
    'fp16': 'True',
    'per_gpu_train_batch_size': 16,
    'per_gpu_eval_batch_size': 16,
    'max_seq_length': 384,
    'doc_stride': 128,
    'max_query_length': 64,
    'learning_rate': 5e-5,
    'num_train_epochs': 2,
    #'max_steps': 100, # If > 0: set total number of training steps to perform. Override num_train_epochs.
    'save_steps': 1000, 
}

# metric definition to extract the results
metric_definitions=[
     {"Name": "train_runtime", "Regex": "train_runtime.*=\D*(.*?)$"},
     {'Name': 'train_samples_per_second', 'Regex': "train_samples_per_second.*=\D*(.*?)$"},
     {'Name': 'epoch', 'Regex': "epoch.*=\D*(.*?)$"},
     {'Name': 'f1', 'Regex': "f1.*=\D*(.*?)$"},
     {'Name': 'exact_match', 'Regex': "exact_match.*=\D*(.*?)$"}]

## Creating an Estimator and start a training job

In [ ]:
# estimator

huggingface_estimator = HuggingFace(
    entry_point='run_squad.py',
    source_dir='./scripts',
    metric_definitions=metric_definitions,
    instance_type='ml.p3.8xlarge',
    instance_count=1,
    volume_size=200,
    role=role,
    transformers_version='4.6',
    pytorch_version='1.7',
    py_version='py36',
    hyperparameters=hyperparameters
)

In [ ]:
# starting the train job with our uploaded datasets as input
huggingface_estimator.fit({'train': input_train, 'validation': input_validation})

# ml.p3.8xlarge, 2 epochでの実行時間の目安
# Training seconds: 758
# Billable seconds: 758

## Estimator Parameters

In [ ]:
# container image used for training job
print(f"container image used for training job: \n{huggingface_estimator.image_uri}\n")

# s3 uri where the trained model is located
print(f"s3 uri where the trained model is located: \n{huggingface_estimator.model_data}\n")

# latest training job name for this estimator
print(f"latest training job name for this estimator: \n{huggingface_estimator.latest_training_job.name}\n")

In [ ]:
# access the logs of the training job
huggingface_estimator.sagemaker_session.logs_for_job(huggingface_estimator.latest_training_job.name)

## Download-fine-tuned-model-from-s3

In [ ]:
import os

OUTPUT_DIR = './output/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [ ]:
from sagemaker.s3 import S3Downloader

# 学習したモデルのダウンロード
S3Downloader.download(
    s3_uri=huggingface_estimator.model_data, # s3 uri where the trained model is located
    local_path='.', # local path where *.targ.gz is saved
    sagemaker_session=sess # sagemaker session used for training the model
)

In [ ]:
# OUTPUT_DIRに解凍します

!tar -zxvf model.tar.gz -C output

## Question Answering on Local

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained('./output')  
tokenizer = AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking') 

以下のセルは`./DDQA-1.0/RC-QA/DDQA-1.0_RC-QA_dev.json`からコピーしたものです

In [ ]:
context = '実は先週、ＣＢＲ６００ＲＲで事故りました。たまにはＣＢＲにも乗らなきゃなーと思い久々にＣＢＲで出勤したところ、家から１０分ほど走ったところにある片側一車線の交差点で対向右折車と衝突してしまいました。自分が直進青信号で交差点へ進入したところで対向右折車線の車が突然右折を開始。とっさに急ブレーキはかけましたが、止まることはできずに右折車に衝突、自分は空中で一回転して左斜め数メートル先の路上へと飛ばされました。'
question='何に乗っていて事故りましたか？'

In [ ]:
#context = 'まぁ，何回か改正してるわけで，自転車を走らせる領域を変更しないって言うのは，怠慢っていうか責任逃れっていうか，道交法に携わってるヤツはみんな馬鹿なのか．大体の人はここまで極端な意見ではないだろうけど，自転車は歩道を走るほうが自然だとは考えているだろう．というのも， みんな自転車乗ってる時歩道を走るでしょ？自転車で歩道走ってても歩行者にそこまで危険な目に合わせないと考えているし，車道に出たら明らかに危険な目に合うと考えている．'
#question='大体の人は自転車はどこを走るのが自然だと思っている？'

In [ ]:
#context = '幸いけが人が出なくて良かったものの、タイヤの脱落事故が後を絶たない。先日も高速道路でトラックのタイヤがはずれ、中央分離帯を越え、反対車線を通行していた観光バスに直撃した。不幸にもバスを運転していた運転手さんがお亡くなりになった。もし、僕がこんな場面に遭遇していたら、この運転手さんのように、乗客の安全を考えて冷静に止まっただろうか？'
#question = '後を絶たないのは何ですか？'

In [ ]:
#context = '右折待ちの一般ドライバーの方は、直進車線からの右折タクシーに驚いて右折のタイミングを失ってしまい、更なる混雑を招いているようでした」と述べていました。２００４年８月６日付けには、ある女性が「道を譲っても挨拶をしない人が多い。特に女性の方。そのため意地悪ですが対向車のドライバーが女性だと譲りません。私はまだ人間が出来ていないので受け流すことが出来ません」ということを言っていましたが、その気持ち良く分かります。私は横断歩道の歩行者に対しては特別真面目で、歩行者がいるかどうかを常に注意して、いるときは必ず止まるよう心掛けています。それでも気付かずに止まることができなかったときは、「ああ、悪いことしちゃったな…」と、バックミラーを見ながら思います。'
#question = '歩行者がいるかどうかを常に注意しているのは誰ですか？'

In [ ]:
# 推論
inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
input_ids = inputs["input_ids"].tolist()[0]
output = model(**inputs)
answer_start = torch.argmax(output.start_logits)  
answer_end = torch.argmax(output.end_logits) + 1 
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

# 結果
print("質問: "+question)
print("回答: "+answer)